In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install openpyxl

In [250]:
# load excel file data
# df = pd.read_excel("~/data/project/pMHC-TCR/seqData/230213_info.xlsx")
file_path = "~/data/project/pMHC-TCR/seqData/230215.xlsx"
df = pd.read_excel(file_path)

In [251]:
df["nSeqHLA"].value_counts()

ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAGGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATGTGAAGGCCCAGTCACAGACTGACCGAGTGGACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTTCTCACACCATCCAGATAATGTATGGCTGCGACGTGGGGCCGGACGGGCGCTTACTCCGCGGGTACCGGCAGGACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCTTGGACCGCGGCGGACATGGCAGCTCAGATCACCAAGCGCAAGTGGGAGGCGGCCCATGCGGCGGAGCAGCAGAGAGCCTACCTGGAGGGCCGGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCACGGACCCCCCCAAGACACATATGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGACCAGACCCAGGACACGGAGCTCGTGGAGACCAGGCCTGCAGGGGATGGAACCTTCCAGAAGTGGGCGGCTGTGGTGGTGCCTTCTGGAGAGGAGCAGAGATACACCTGCCATGTGCAGCATGAGGGTCTGCCCAAGCCCCTCACCCTGAGATGGGAGCTGTCTTCCCAGCCCACCATCCCCATCGTGGGCATCATTGCTGGCCTGGTTCTCCTTGGAGCTGTGATCACTGGAGCTGTGGTCGCTGCCGTGATGTGGAGGA

In [256]:
# find the longest length of aaSeqCDR1
for i in ["TRA", "TRB"]:
    for seq in ["aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]:
        print(i, seq, df.loc[df["chain"] == i, seq].str.len().max())
    # df.loc[df['chain'] == 'TRA', 'aaSeqCDR1'].str.len().max()

TRA aaSeqCDR1 7
TRA aaSeqCDR2 8
TRA aaSeqCDR3 25
TRB aaSeqCDR1 6
TRB aaSeqCDR2 7
TRB aaSeqCDR3 21


In [150]:
# reindex the df by cellname and chain
# df.sort_values(by=['cellname', "chain"])
df = df.set_index(['cellname', "chain"])
# extract the NeoAA, HLA, and aaSeqCDR columns
df = df[["NeoAA", "HLA", "aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]]
# df["aaSeqCDR1"].value_counts()
df["aaSeqCDR"] = df[df.columns[2:]].apply(
    # for TRA aaSeqCDR1, padding with 'X' to make it 7 aa long
    lambda x: x[0] + 'X' * (7 - len(x[0])) + x[1] + x[2],
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)

In [151]:
idx = pd.IndexSlice

df_a = df.loc[idx[:,"TRA"],]
df_a["AseqCDR"] = df_a["aaSeqCDR"]
df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_a.index = df_a.index.droplevel(1)
# print(df_a)
df_b = df.loc[idx[:,"TRB"],]
df_b["BseqCDR"] = df_b["aaSeqCDR"]
df_b.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_b.index = df_b.index.droplevel(1)
# print(df_b)

# merge the TRA and TRB dataframes by cellname, HLAs, and NeoAA
df_ab = pd.merge(df_a, df_b, on=["cellname", "HLA", "NeoAA"])
df_ab
# df_ab = pd.merge(df_a, df_b, on="cellname")
# df_ab

/tmp/ipykernel_4163421/3989028287.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["AseqCDR"] = df_a["aaSeqCDR"]
/tmp/ipykernel_4163421/3989028287.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
/tmp/ipykernel_4163421/3989028287.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


In [152]:
df_ab.to_csv(file_path.replace(".xlsx", ".csv"))

In [85]:
# not used
# (df['aaSeqCDR'].value_counts() > 1).sum()
count = 0
for cell in df.index:
    rep_dict = {}
    if df.loc[(cell), "aaSeqCDR"] in rep_dict:
        count += 1
    else:
        rep_dict[df.loc[(cell), "aaSeqCDR"]] = 1
print(rep_dict)
print(count)

{'SGHTA,FQGNSA,CASSLGVYEQYF': 1}
0


In [47]:
# not used
# sorting data: combine the sequence from the same cell (has the same cellname), 
# add the prefix of the column chain to the columns CDR1, CDR2, CDR3
data = df.groupby("cellname").agg(
    {
        "NeoGeneID": "first",
        "NeoAA" : "first",
        "HLA": "first",
        "chain": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR1": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR2": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR3": lambda x: x.iloc[0] + " " + x.iloc[1],
    }
)
# split each aaSeqCDR1 into 2 columns, one for A chain and one for B chain
data[['aaSeqCDR1_B', 'aaSeqCDR1_A']] = data.aaSeqCDR1.str.split(" ",expand=True,)
data[['aaSeqCDR2_B', 'aaSeqCDR2_A']] = data.aaSeqCDR2.str.split(" ",expand=True,)
data[['aaSeqCDR3_B', 'aaSeqCDR3_A']] = data.aaSeqCDR3.str.split(" ",expand=True,)
data.drop(columns=['aaSeqCDR1', 'aaSeqCDR2', 'aaSeqCDR3'], inplace=True)

In [135]:
# not used
df.groupby(['aaSeqCDR', 'chain', 'cellname']).agg(
    {
        # find the TRA and TRB chains of the same cell has the same aaSeqCDR
        'aaSeqCDR': 'count',
    }
)

In [153]:
df_ab

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


# Load the dataset into torch dataset

In [25]:
# not used
# torch dataset
class TCRDataset(Dataset):
    '''
    Use each two rows of data as a sample (one for alpha chain, one for beta chain), each sample has the common cellname
    Different from the chain, the HLA class is the same for each sample
    The aaSeqCDR1, aaSeqCDR2, aaSeqCDR3 are the CDR1, CDR2, CDR3 of the alpha chain and beta chain, respectively.
    '''
    def __init__(self):
        self.root = "~/data/project/pMHC-TCR/seqData/230213_info.csv"
        

In [191]:
file_path = "~/data/project/pMHC-TCR/seqData/230215.csv"
datasets = pd.read_csv(file_path)
# combine the AseqCDR and BseqCDR into one column with _ as the separator
datasets["seqCDR"] = datasets["AseqCDR"] + "_" + datasets["BseqCDR"]
# drop the AseqCDR and BseqCDR columns
datasets.drop(columns=["AseqCDR", "BseqCDR"], inplace=True)
datasets.index = datasets["cellname"]
datasets.drop(columns=["cellname"], inplace=True)
datasets

,NeoAA,HLA,seqCDR
cellname,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF_SGHTA_FQGNSA_...
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF_LGHNT_FRNRAP_CAS...
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF_KGHSH_LQKENI_C...
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF_DFQATT_SNEGSKA_CS...
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF_MNHEY_SMNVEV_CASSL...
...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF_SGHAT_FQNNGV_CA...
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF_SGHNS_FNNNVP_CAS...
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF_PRHDT_FYEKMQ_CASSP...


In [192]:
# select the first 3-peptide and last 3-peptide of NeoAA sequence
# datasets["first_3"] = datasets["NeoAA"].str.slice(0,3)
# datasets["last_3"] = datasets["NeoAA"].str.slice(-4,-1)
datasets["NeoAA"] = datasets["NeoAA"].str.slice(0,3) + "_" + datasets["NeoAA"].str.slice(-4,-1)

In [194]:
datasets["NeoAA"].value_counts()

SSC_MNQ    421
VVG_GVG    167
ATA_LSG    128
VLL_LSY     64
VVV_DVG     40
VVG_DVG     21
TTA_LSG     14
VVV_GVG     14
SLM_IPH      8
LVT_LLT      8
SVC_ILS      1
Name: NeoAA, dtype: int64

We check the first 3-peptide and last 3-peptide of the neoantigen sequence, there are only 11 cases. So I think that they could be determined as one-hot encode.

In [227]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = datasets.drop("HLA", axis=1).values
y = datasets.iloc[:,1].values

labelencoder_x = LabelEncoder()
X[:,0] = labelencoder_x.fit_transform(X[:,0])
onehotEncoder = OneHotEncoder()
X_feature = onehotEncoder.fit_transform(X[:,0].reshape(-1,1)).toarray()
X_feature.shape, X[:,1].shape
X = np.hstack((X_feature, X[:,1].reshape(-1,1)))

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

## Split into train set and test set

In [236]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [239]:
X_train.shape, X_test.shape

((708, 12), (178, 12))

## Loading the Atchley factor

In [240]:
af = pd.read_excel("~/data/project/pMHC-TCR/library/Atchley_factors.xlsx")
af.to_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv", index=False)
af = pd.read_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv")

In [242]:
af.index = af["Amino acid"]
af.drop(columns=["Amino acid"], inplace=True)
af

,Factor I,Factor II,Factor III,Factor IV,Factor V
Amino acid,,,,,
A,-0.591,-1.302,-0.733,1.570,-0.146
C,-1.343,0.465,-0.862,-1.020,-0.255
D,1.050,0.302,-3.656,-0.259,-3.242
E,1.357,-1.453,1.477,0.113,-0.837
F,-1.006,-0.590,1.891,-0.397,0.412
G,-0.384,1.652,1.330,1.045,2.064
H,0.336,-0.417,-1.673,-1.474,-0.078
I,-1.239,-0.547,2.131,0.393,0.816
K,1.831,-0.561,0.533,-0.277,1.648


In [262]:
# Use the Atchley factors to encode the aaSeqCDR, and use t to represent the _ in the sequence
# X[:,-1][0]
# 'TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF_SGHTA_FQGNSA_CASSLGVYEQYF'

def encode_seqCDR(seqCDR):
    seqCDR = seqCDR.replace("_", "")
    encoding_list = []
    for i in range(len(seqCDR)):
        encoding_list.append(af.loc[seqCDR[i]])
    return np.array(encoding_list)

In [266]:
# encode_seqCDR(X[:,-1][0]).shape
for i in range(len(X[:,-1])):
    input_seq = X[i, -1].split("_")
    # store the encoding result of each aaSeqCDR
    encode_seq_result = []
    for i in input_seq:
        encode_seq_result.append(encode_seqCDR(i))

(7, 5)
(8, 5)
(15, 5)
(5, 5)
(6, 5)
(12, 5)


In [ ]:
# create a LSTM model, the input is the encoding result of each aaSeqCDR
# the input size is depend on the length of the encoding result
# the output is the HLA class of the sample
# the output of the LSTM is the probability of the HLA class
# the latent variable is the hidden state of the LSTM, and we set it to 5
seq_len = 
rnn = nn.LSTM(seq_len, hidden_size)
input = torch.randn(10, )